# Construction scenario maps

Maps of the results of the construction scenarios.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import shapely
import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch
from census import Census
import os
from glob import glob

In [ ]:
plt.style.use('asu-light')

In [ ]:
capi = Census(os.environ['CENSUS_API_KEY'])

In [ ]:
base_npvs = pd.read_parquet('../data/Current appreciation_net_present_value.parquet')

In [ ]:
# figure out which were vacant (needed to calculate unit change)
gid_info = pd.read_sql('SELECT gid, tract, building_propertylandusestndcode FROM diss.gp16', 'postgresql://matthewc@localhost/matthewc')

In [ ]:
base_npvs = base_npvs.merge(gid_info, left_index=True, right_on='gid', how='left', validate='1:1')
base_npvs['vacant'] = base_npvs.building_propertylandusestndcode == 'VL101'

In [ ]:
def calculate_redevelopment (npvs):
    npvs = npvs.copy()
    npvs['most_profitable'] = npvs[['duplex', 'sfh', 'sixplex', 'threeplex', 'existing']].idxmax(axis=1)
    
    assert not (npvs.most_profitable == 'sfh').any()  # this assumption is important when computing effects for people in new housing

    npvs = npvs.merge(gid_info, left_index=True, right_on='gid', how='left', validate='1:1')

    assert not npvs.building_propertylandusestndcode.isnull().any()

    npvs['vacant'] = npvs.building_propertylandusestndcode == 'VL101'

    # one less unit to account for teardown
    npvs['new_units'] = npvs.most_profitable.replace({
        'existing': 0,
        'sfh': 1,
        'duplex': 2,
        'threeplex': 3,
        'sixplex': 6
    })

    npvs['destroyed_units'] = 0
    # one unit assumed destroyed, except on vacant lots
    npvs.loc[~npvs.vacant & (npvs.most_profitable != 'existing'), 'destroyed_units'] = 1

    # create table
    row = {
        'Non-redeveloped parcels': (npvs.most_profitable == 'existing').sum(),
        'Non-redeveloped parcels %': '{:.1f}%'.format((npvs.most_profitable == 'existing').mean() * 100),
        'Single-family home': npvs.loc[npvs.most_profitable == 'sfh', 'new_units'].sum(),
        'Duplex': npvs.loc[npvs.most_profitable == 'duplex', 'new_units'].sum(),
        'Threeplex': npvs.loc[npvs.most_profitable == 'threeplex', 'new_units'].sum(),
        'Sixplex': npvs.loc[npvs.most_profitable == 'sixplex', 'new_units'].sum(),
        'Total': npvs.new_units.sum(),
        'Teardowns': npvs.destroyed_units.sum(),
        'Marginal units': npvs.new_units.sum() - npvs.destroyed_units.sum()
    }
    
    npvs['marginal_units'] = npvs.new_units - npvs.destroyed_units
    by_tract = npvs.groupby('tract').marginal_units.sum()
    
    return row, by_tract

In [ ]:
rows = {}
tract_totals = {}

for npvf in glob('../data/*_net_present_value.parquet'):
    scenario = npvf[8:-26].replace('_hqta', ' (HQTA)').replace('Base', 'Low appreciation')
    print(scenario)
    rows[scenario], tract_totals[scenario] = calculate_redevelopment(pd.read_parquet(npvf))

In [ ]:
order = [
    'Current appreciation',
    'Low appreciation',
    'Low operating cost (25%)',
    'High construction cost',
    'Low discount rate',
    'Equal discount rate (8% existing and new)'
]

result_table = pd.DataFrame(rows).transpose().loc[[
    *order,
    *map('{} (HQTA)'.format, order)
]]

result_table.to_parquet('../data/profitability_table.parquet')

for col in result_table.columns:
    if col != 'Non-redeveloped parcels %':
        result_table[col] = (result_table[col].astype('int64') / 1000).round().astype('int64').apply('{:,d}'.format)

result_table.loc['Total candidate parcels', 'Non-redeveloped parcels %'] = len(base_npvs)
result_table

In [ ]:
print(result_table.drop(columns=['Non-redeveloped parcels']).fillna('').to_latex())

In [ ]:
# hacky, but just copied manually from the property sales notebook
non_hqta_sales = pd.Series({
    'Low appreciation': 2,
    'Current appreciation': 45,
    'Equal discount rate (8%)': 190,
    'High construction cost': 0,
    'Low discount rate': 36,
    'Low operating cost': 644
}).sort_values()
plt.barh(-np.arange(len(non_hqta_sales)), non_hqta_sales)
plt.yticks(-np.arange(len(non_hqta_sales)), non_hqta_sales.index)
plt.xlabel('Marginal new units (thousands)')
plt.savefig('../../defense/const_sales.pdf', bbox_inches='tight')

In [ ]:
non_hqta_sales

## Maps

In [ ]:
tracts = gp.read_postgis("SELECT gid, statefp, countyfp, tractce, aland, geog::geometry as geom FROM diss.ca_tracts WHERE countyfp IN ('025', '037', '059', '065', '071', '111')", 'postgresql://matthewc@localhost/matthewc')

In [ ]:
tracts['geoid'] = tracts.statefp.str.cat(tracts.countyfp).str.cat(tracts.tractce)

### Housing unit counts by tract for normalization

In [ ]:
hu_count = base_npvs.groupby('tract').size()

In [ ]:
hu_count = capi.acs5.state_county_tract(['B25001_001E'], '06', Census.ALL, Census.ALL, year=2017)

In [ ]:
hu_count = pd.DataFrame(hu_count)

In [ ]:
hu_count['geoid'] = hu_count.state.str.cat(hu_count.county).str.cat(hu_count.tract)
hu_count = hu_count.set_index('geoid')

In [ ]:
tracts = tracts.to_crs(epsg=26911).set_index('geoid')

In [ ]:
# TODO don't we need to refer to the column name here?
tracts['hu_count'] = hu_count.B25001_001E.reindex(tracts.index, fill_value=0)

In [ ]:
hqta = gp.read_postgis('SELECT geog::geometry as geom FROM diss.hqta', 'postgresql://matthewc@localhost/matthewc').to_crs(26911)

land = gp.read_file('../../sorting/data/ne_10m_land.shp').to_crs(epsg=26911)

roads = pd.concat([gp.read_file(i).to_crs(epsg=26911) for i in glob('../../sorting/data/tl_roads/*.shp')], ignore_index=True)

counties = gp.read_file('../../sorting/data/counties/tl_2019_us_county.shp').to_crs(26911)
counties = counties[(counties.STATEFP == '06') & counties.NAME.isin(['Los Angeles', 'Ventura', 'Orange', 'Riverside', 'San Bernardino', 'Imperial'])]

In [ ]:
colors = {
    (0, 0): ('0', '#f8f8f8'),
    (0, 1): ('< 1', '#b2e2e2'),
    (1, 5): ('1–5', '#66c2a4'),
    (5, 25): ('5–25', '#2ca25f'),
    (25, np.inf): ('≥25', '#006d2c')
}

def color_for_val (val):
    if pd.isnull(val):
        return '#ffffff'
    
    for rnge, spec in colors.items():
        if rnge[0] == rnge[1] and val == rnge[0]:
            return spec[1]
        elif val >= rnge[0] and val < rnge[1]:
            return spec[1]
    else:
        raise ValueError(f'Value {val} not in any range!')

def map_const (dev_totals, ax=None, draw_map=True, draw_hqta=False, legend=True, inset=True, _inset=False):
    if ax is None:
        f, ax = plt.subplots(figsize=(10, 10))
        
    if draw_map:
        tract_development = tracts.copy()
        tract_development['marginal_units'] = dev_totals.reindex(tract_development.index, fill_value=0)
        tract_development['marginal_units_per_sq_km'] = tract_development.marginal_units / tract_development.aland * 1000**2#.hu_count.replace({0: np.nan}) * 100
        tract_development.to_crs(epsg=26911).plot(ax=ax, color=tract_development.marginal_units_per_sq_km.apply(color_for_val))
        roads.plot(color='#888888', ax=ax, lw=0.25)
        counties.plot(edgecolor='#000',  facecolor='none', ax=ax, lw=1)
        
        if draw_hqta:
            hqta.plot(ax=ax, color='#00a3e0', alpha=0.3, lw=1)
        
        #water.plot(color='#aaaaaa', ax=ax)
        if _inset:
            ax.set_ylim(3.73e6, 3.79e6)
            ax.set_xlim(3.1e5, 4.2e5)
        elif draw_hqta:
            ax.set_ylim(3.67e6, 3.83e6)
            ax.set_xlim(3.45e5, 5.8e5)
        else:
            ax.set_ylim(3.59e6, 3.98e6)
            ax.set_xlim(2.75e5, 7.7e5)

        if inset and not _inset:
            inset_ax = ax.inset_axes([0.525, 0.6, 0.5, 0.4])
            inset_ax.set_xlabel('Central Los Angeles')
            map_const(dev_totals, draw_hqta=draw_hqta, ax=inset_ax, legend=False, _inset=True)
            
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_yticks([])
    ax.set_xticks([])

    ax.set_aspect('equal')

    if legend:
        patches = [mpatch.Patch(color=c[1]) for c in colors.values()]
        labels = [c[0] for c in colors.values()]
        
        if draw_hqta:
            patches.append(mpatch.Patch(color='#00a3e0', alpha=0.3))
            labels.append('High-quality transit area')
        
        ax.legend(
            patches,
            labels,
            loc='lower left' if draw_map else 'center',
            title='Marginal new housing units per sq. km.',
            framealpha=1,
            fontsize='medium' if draw_map else 'large',
            title_fontsize='medium' if draw_map else 'large'
        )
        
        if not draw_map:
            ax.set_axis_off()

    if draw_map:
        return tract_development    

In [ ]:
base_growth = map_const(tract_totals['Current appreciation'])
plt.savefig('../../dissertation/fig/construction/unit_growth_current_app.png', dpi=300, bbox_inches='tight')

In [ ]:
base_growth = map_const(tract_totals['Current appreciation (HQTA)'], draw_hqta=True, inset=False)
plt.savefig('../../dissertation/fig/construction/unit_growth_current_app_hqta.png', dpi=300, bbox_inches='tight')

In [ ]:
base_growth = map_const(tract_totals['Low appreciation'])
plt.savefig('../../dissertation/fig/construction/unit_growth_low_app.png', dpi=300, bbox_inches='tight')

In [ ]:
base_growth = map_const(tract_totals['Low operating cost (25%)'])
plt.savefig('../../dissertation/fig/construction/unit_growth_low_opcost.png', dpi=300, bbox_inches='tight')

In [ ]:
tract_totals.keys()

In [ ]:
# sensitivity test maps
f, axs = plt.subplots(3, 2, figsize=(12, 16))
axs = axs.reshape(-1)

for scenario, ax in zip(['Current appreciation', *sorted([k for k in tract_totals.keys() if not 'HQTA' in k and not 'Current appreciation' in k])], axs[:-1]):
    map_const(tract_totals[scenario], ax=ax, inset=False, legend=False)
    ax.set_title(scenario)

map_const(None, draw_map=False, legend=True, ax=axs[-1]) # draw legend
plt.savefig('../../dissertation/fig/construction/unit_growth_sensitivity.png', bbox_inches='tight', dpi=600)

In [ ]:
# sensitivity test maps
f, axs = plt.subplots(3, 2, figsize=(12, 16))
axs = axs.reshape(-1)

for scenario, ax in zip(['Current appreciation (HQTA)', *sorted([k for k in tract_totals.keys() if 'HQTA' in k and not 'Current appreciation' in k])], axs[:-1]):
    map_const(tract_totals[scenario], ax=ax, draw_hqta=True, inset=False, legend=False)
    ax.set_title(scenario)

map_const(None, draw_map=False, legend=True, ax=axs[-1], draw_hqta=True) # draw legend
plt.savefig('../../dissertation/fig/construction/unit_growth_sensitivity_hqta.png', bbox_inches='tight', dpi=600)

## Maps of the sales scenarios

In [ ]:
sales_scenarios = pd.read_parquet('../data/npv_tract_scenarios.parquet').reset_index()

sales_scenarios[['geoid', 'sfmf', 'age', 'tenure']] = sales_scenarios['index'].str.split('_', expand=True)

marginal_units = sales_scenarios.drop(columns=['index', 'sfmf', 'age', 'tenure']).groupby('geoid').sum()
marginal_units.head()

In [ ]:
# sales model maps
f, axs = plt.subplots(3, 2, figsize=(12, 16))
axs = axs.reshape(-1)

map_const(marginal_units.npv_current_appreciation, ax=axs[0], draw_hqta=False, inset=False, legend=False)
axs[0].set_title('Current appreciation')

map_const(marginal_units.npv_current_appreciation_hqta, ax=axs[1], draw_hqta=False, inset=False, legend=False)
axs[1].set_title('Current appreciation (HQTA)')

map_const(marginal_units.npv_low_opcost, ax=axs[2], draw_hqta=False, inset=False, legend=False)
axs[2].set_title('Low operating cost')

map_const(marginal_units.npv_low_opcost_hqta, ax=axs[3], draw_hqta=False, inset=False, legend=False)
axs[3].set_title('Low operating cost (HQTA)')

map_const(marginal_units.npv_base, ax=axs[4], draw_hqta=False, inset=False, legend=False)
axs[4].set_title('Low appreciation')

# map_const(marginal_units.npv_base_hqta, ax=axs[4], draw_hqta=False, inset=False, legend=False)
# axs[4].set_title('Low appreciation (HQTA)')

map_const(None, draw_map=False, legend=True, ax=axs[5], draw_hqta=False) # draw legend
plt.savefig('../../dissertation/fig/sales/sales_maps.png', bbox_inches='tight', dpi=600)

In [ ]:
base_npvs['profitability'] = base_npvs[['sfh', 'duplex', 'threeplex', 'sixplex']].max(axis=1) / base_npvs.existing
tract_pft_mean = np.minimum(base_npvs.loc[base_npvs.profitability > 1].groupby('tract').profitability.median(), 3)

In [ ]:
tracts['profitability'] = tract_pft_mean.reindex(tracts.index)

In [ ]:
tracts.plot(column='profitability', legend=True)